In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn import *
from scipy.stats import *
from framework.data import sql

/home/ravirao/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/home/ravirao/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [ ]:
data = pd.read_sql('''
select `Provider Number`, `Primary specialty`, `Graduation year`, `Total Performance Score`
from hospital_practitioners
''', sql)
print(data.shape)

In [16]:
f = open('results.csv', 'w')
print('Specialty', 'Spearman Value', 'Spearman PValue', 'R', 'R^2', 'Slope', 'Mean', 'Standard Deviation', sep=',', file=f)
f.flush()
for specialty, ind in data.groupby('Primary specialty').groups.items():
    g = data.ix[ind, :].groupby('Graduation year')
    year = np.array(list(g.groups.keys()))
    years = year[:, np.newaxis]
    std_score = g['Total Performance Score'].std()
    mean_score = g['Total Performance Score'].mean()
    
    lr = linear_model.LinearRegression()
    lr.fit(years, mean_score)

    r2 = lr.score(years, mean_score)
    r = r2**(1/2)
    m = lr.coef_[0]
    try:
        shapiro_val = shapiro(mean_score)
    except:
        shapiro_val = None
    spearman_val = spearmanr(years, mean_score)
    
    print(specialty,
          '%0.3f' % (spearman_val.correlation),
          '%0.5f' % (spearman_val.pvalue),
          '%0.3f' % (r),
          '%0.3f' % (r**2),
          '%0.3f' % (m),
          '%0.3f' % (mean_score.mean()),
          '%0.3f' % (mean_score.std()),
          sep=',',
          file=f)
    f.flush()
f.close()